<a href="https://colab.research.google.com/github/Ishlafakhri/Leads-Scoring-Model/blob/main/Leads_Scoring_Using_Machine_Learning_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Leads Scoring Model

##Data Preparation

###Import Library and Dataset

In [1]:
# Data manipulation libraries
import pandas as pd
import numpy as np

# Preprocessing libraries
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder

# Model selection libraries
from sklearn.model_selection import train_test_split, cross_val_score

# Model building libraries
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
import xgboost as xgb
from xgboost import XGBClassifier  # Direct import of XGBClassifier for convenience

# Model evaluation libraries
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Visualization libraries
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [2]:
#Import Data from Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Load the dataset from Google Drive and store it in a DataFrame.
# Dataset from kaggle: https://www.kaggle.com/datasets/amritachatterjee09/lead-scoring-dataset

df = pd.read_csv('/content/drive/MyDrive/Project Data Science/Lead Scoring.csv')

In [4]:
# Set the display option to show all columns when displaying a DataFrame.
pd.set_option('display.max_columns', None)
df.head()

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,Specialization,How did you hear about X Education,What is your current occupation,What matters most to you in choosing a course,Search,Magazine,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Receive More Updates About Our Courses,Tags,Lead Quality,Update me on Supply Chain Content,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,Page Visited on Website,NaN,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Interested in other courses,Low in Relevance,No,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,Email Opened,India,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Ringing,NaN,No,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,Email Opened,India,Business Administration,Select,Student,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,Might be,No,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,Unreachable,India,Media and Advertising,Word Of Mouth,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Ringing,Not Sure,No,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,Converted to Lead,India,Select,Other,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,Might be,No,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified


##Data Cleaning

###Check Missing Values

In [5]:
# Display information about the DataFrame to understand the data types and number of non-null values in each column.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 37 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Prospect ID                                    9240 non-null   object 
 1   Lead Number                                    9240 non-null   int64  
 2   Lead Origin                                    9240 non-null   object 
 3   Lead Source                                    9204 non-null   object 
 4   Do Not Email                                   9240 non-null   object 
 5   Do Not Call                                    9240 non-null   object 
 6   Converted                                      9240 non-null   int64  
 7   TotalVisits                                    9103 non-null   float64
 8   Total Time Spent on Website                    9240 non-null   int64  
 9   Page Views Per Visit                           9103 

In [6]:
# Check for missing values
print(df.isnull().sum())

Prospect ID                                         0
Lead Number                                         0
Lead Origin                                         0
Lead Source                                        36
Do Not Email                                        0
Do Not Call                                         0
Converted                                           0
TotalVisits                                       137
Total Time Spent on Website                         0
Page Views Per Visit                              137
Last Activity                                     103
Country                                          2461
Specialization                                   1438
How did you hear about X Education               2207
What is your current occupation                  2690
What matters most to you in choosing a course    2709
Search                                              0
Magazine                                            0
Newspaper Article           

###Fill Missing Values

In [7]:
# Identify numeric columns
numeric_cols = df.select_dtypes(include=['int64', 'float64'])

# Fill missing numeric data with their mean
df[numeric_cols.columns] = numeric_cols.fillna(numeric_cols.mean())

# Identify non-numeric columns
non_numeric_cols = df.select_dtypes(exclude=['int64', 'float64'])

# Fill missing non-numeric data with 'missing'
df[non_numeric_cols.columns] = non_numeric_cols.fillna('Missing')

In [8]:
# Check for missing values
print(df.isnull().sum())

Prospect ID                                      0
Lead Number                                      0
Lead Origin                                      0
Lead Source                                      0
Do Not Email                                     0
Do Not Call                                      0
Converted                                        0
TotalVisits                                      0
Total Time Spent on Website                      0
Page Views Per Visit                             0
Last Activity                                    0
Country                                          0
Specialization                                   0
How did you hear about X Education               0
What is your current occupation                  0
What matters most to you in choosing a course    0
Search                                           0
Magazine                                         0
Newspaper Article                                0
X Education Forums             

In [9]:
# Display the first few rows of the DataFrame.
df.head()

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,Specialization,How did you hear about X Education,What is your current occupation,What matters most to you in choosing a course,Search,Magazine,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Receive More Updates About Our Courses,Tags,Lead Quality,Update me on Supply Chain Content,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0,0.0,0,0.0,Page Visited on Website,Missing,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Interested in other courses,Low in Relevance,No,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,0,5.0,674,2.5,Email Opened,India,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Ringing,Missing,No,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,1,2.0,1532,2.0,Email Opened,India,Business Administration,Select,Student,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,Might be,No,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,0,1.0,305,1.0,Unreachable,India,Media and Advertising,Word Of Mouth,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Ringing,Not Sure,No,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,1,2.0,1428,1.0,Converted to Lead,India,Select,Other,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,Might be,No,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified


##Feature Selection and Engineering

###Feature Encoding

In [10]:
# Define a function to encode non-numeric data types using one-hot encoding or label encoding.

def encode_non_numeric(df):
    """
    Encodes non-numeric columns in the DataFrame using label encoding.

    Parameters:
        df (pd.DataFrame): The DataFrame to encode.

    Returns:
        pd.DataFrame: A DataFrame with non-numeric columns encoded.
    """
    # Create a copy of the DataFrame to avoid modifying the original data
    encoded_df = df.copy()

    # Identify non-numeric columns
    non_numeric_cols = encoded_df.select_dtypes(exclude=['int64', 'float64'])

    # Initialize LabelEncoder
    le = LabelEncoder()

    # Apply LabelEncoder to each non-numeric column
    for column in non_numeric_cols.columns:
        # Fit and transform the data
        # Use `astype(str)` to ensure proper conversion for mixed types
        encoded_df[column] = le.fit_transform(non_numeric_cols[column].astype(str))

    return encoded_df

In [11]:
# Apply the encoding function to the DataFrame and store the result.
df_encode = encode_non_numeric(df)

In [12]:
# Display the encoded DataFrame.
df_encode

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,Converted,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,Specialization,How did you hear about X Education,What is your current occupation,What matters most to you in choosing a course,Search,Magazine,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Receive More Updates About Our Courses,Tags,Lead Quality,Update me on Supply Chain Content,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity
0,4427,660737,0,7,0,0,0,0.0,0,0.00,11,19,16,7,5,0,0,0,0,0,0,0,0,0,8,1,0,0,5,5,1,1,15.0,15.0,0,0,8
1,1554,660728,0,8,0,0,0,5.0,674,2.50,5,12,16,7,5,0,0,0,0,0,0,0,0,0,15,3,0,0,5,5,1,1,15.0,15.0,0,0,4
2,5140,660727,1,1,0,0,1,2.0,1532,2.00,5,12,1,7,4,0,0,0,0,0,0,0,0,0,20,2,0,0,4,1,1,0,14.0,20.0,0,1,4
3,467,660719,1,1,0,0,0,1.0,305,1.00,14,12,11,10,5,0,0,0,0,0,0,0,0,0,15,4,0,0,5,1,1,0,13.0,17.0,0,0,8
4,1874,660681,1,3,0,0,1,2.0,1428,1.00,1,12,16,5,5,0,0,0,0,0,0,0,0,0,20,2,0,0,5,1,1,0,15.0,18.0,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9235,951,579564,1,1,1,0,1,8.0,1845,2.67,4,26,8,7,5,0,0,0,0,0,0,0,0,0,20,0,0,0,4,1,1,0,15.0,17.0,0,0,3
9236,4782,579546,1,1,0,0,0,2.0,238,2.00,13,12,11,7,5,0,0,0,0,0,0,0,0,0,26,2,0,0,4,1,1,0,14.0,19.0,0,1,12
9237,6184,579545,1,1,1,0,0,2.0,199,2.00,13,12,1,7,5,0,0,0,0,0,0,0,0,0,22,4,0,0,4,1,1,0,13.0,20.0,0,1,12
9238,3055,579538,1,3,0,0,1,3.0,499,3.00,13,12,7,4,2,2,0,0,0,0,0,0,0,0,12,3,0,0,2,4,1,1,15.0,16.0,0,0,12


##Data Modelling

###Model Preparation

In [13]:
# Setting 'Lead Number' as the index
df_encode.set_index('Lead Number', inplace=True)

# Dropping unnecessary column 'Prospect ID'
df_encode.drop(['Prospect ID'], axis=1, inplace=True)

# Assuming all categorical data are already encoded and proceeding with train-test split
from sklearn.model_selection import train_test_split

# Split the data
X = df_encode.drop('Converted', axis=1)
y = df_encode['Converted']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Checking the shape of the train and test sets
X_train.shape, X_test.shape

((7392, 34), (1848, 34))

###Model Selection

In [14]:
#Model Selection

def logistic_regression(X_train, y_train):
    model = LogisticRegression()
    model.fit(X_train, y_train)
    return model

def random_forest(X_train, y_train):
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    return model

def gradient_boosting(X_train, y_train):
    model = GradientBoostingClassifier()
    model.fit(X_train, y_train)
    return model

def svm_classifier(X_train, y_train):
    model = SVC()
    model.fit(X_train, y_train)
    return model

def xgboost_classifier(X_train, y_train):
    model = XGBClassifier()
    model.fit(X_train, y_train)
    return model

###Model Evaluation

In [15]:
# Model evaluation function
def evaluate_model(model, X_test, y_test):
    predictions = model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    report = classification_report(y_test, predictions)
    return accuracy, report

In [16]:
# Running and evaluating each model
models = {
    "Logistic Regression": logistic_regression,
    "Random Forest": random_forest,
    "Gradient Boosting": gradient_boosting,
    "SVM": svm_classifier,
    "XGBoost": xgboost_classifier
}

# Compare the models
for name, model_func in models.items():
    print(f"Running {name}...")
    model = model_func(X_train, y_train)
    accuracy, report = evaluate_model(model, X_test, y_test)
    print(f"{name} Accuracy: {accuracy}")
    print(f"Classification Report for {name}:\n{report}\n")

Running Logistic Regression...
Logistic Regression Accuracy: 0.8262987012987013
Classification Report for Logistic Regression:
              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1107
           1       0.82      0.73      0.77       741

    accuracy                           0.83      1848
   macro avg       0.82      0.81      0.82      1848
weighted avg       0.83      0.83      0.82      1848


Running Random Forest...


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Random Forest Accuracy: 0.9296536796536796
Classification Report for Random Forest:
              precision    recall  f1-score   support

           0       0.92      0.96      0.94      1107
           1       0.94      0.88      0.91       741

    accuracy                           0.93      1848
   macro avg       0.93      0.92      0.93      1848
weighted avg       0.93      0.93      0.93      1848


Running Gradient Boosting...
Gradient Boosting Accuracy: 0.9383116883116883
Classification Report for Gradient Boosting:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95      1107
           1       0.95      0.89      0.92       741

    accuracy                           0.94      1848
   macro avg       0.94      0.93      0.94      1848
weighted avg       0.94      0.94      0.94      1848


Running SVM...
SVM Accuracy: 0.7148268398268398
Classification Report for SVM:
              precision    recall  f1-score   support

     

###Fit Data Into Model

In [17]:
# Choose XGBoost for the best model.

# Building the XGBoost model
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# Getting probability predictions and rounding to two decimal places
xgb_prob_predictions = [round(prob, 2) for prob in xgb_model.predict_proba(X_test)[:, 1]]  # Assuming binary classification

# Function to categorize probabilities
def categorize_probability(prob):
    if 0.1 <= prob <= 0.3:
        return 'Low Probability'
    elif 0.4 <= prob <= 0.6:
        return 'Medium Probability'
    elif 0.7 <= prob <= 1.0:
        return 'High Probability'
    else:
        return 'Undefined'  # For probabilities outside the specified ranges

# Apply categorization to probability predictions
probability_categories = [categorize_probability(prob) for prob in xgb_prob_predictions]

# Convert probabilities to 0 or 1 based on threshold using list comprehension
xgb_predictions = [int(prob > 0.5) for prob in xgb_prob_predictions]

# Evaluate the model using the binary predictions
xgb_accuracy = accuracy_score(y_test, xgb_predictions)
xgb_report = classification_report(y_test, xgb_predictions)

# Print the results
print("XGBoost Model Accuracy:", xgb_accuracy)
print("Classification Report:\n", xgb_report)

# Adding predictions to the test set for review
X_test['Predicted_Probability'] = xgb_prob_predictions
X_test['Probability_Category'] = probability_categories
X_test['Predicted_Conversion'] = xgb_predictions
X_test['Actual_Conversion'] = y_test

XGBoost Model Accuracy: 0.9475108225108225
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.97      0.96      1107
           1       0.95      0.92      0.93       741

    accuracy                           0.95      1848
   macro avg       0.95      0.94      0.95      1848
weighted avg       0.95      0.95      0.95      1848



##Model Evaluation

###Confusion Matrix (Model Evaluation)

In [18]:
# Create confusion matrix
xgb_conf_matrix = confusion_matrix(y_test, xgb_predictions)

# Custom labels for x and y axes
classes = ['Not Converted (0)', 'Converted (1)']

# Calculate percentages
total = np.sum(xgb_conf_matrix)
percentages = (xgb_conf_matrix / total * 100).astype(int)

# Create text annotations for each cell
annotation_text = [[f'{value}<br>({percent}%)' for value, percent in zip(row, percentages_row)] for row, percentages_row in zip(xgb_conf_matrix, percentages)]

# Create the heatmap
fig = ff.create_annotated_heatmap(
    xgb_conf_matrix,
    x=classes,
    y=classes,
    colorscale='Viridis',
    showscale=True,
    annotation_text=annotation_text
)

fig.update_layout(
    title='Confusion Matrix for XGBoost Model',
    xaxis=dict(title='Predicted label'),
    yaxis=dict(title='True label'),  # Invert y-axis to have the most important feature at the top
    width=1000,  # Width in pixels
    height=800   # Height in pixels
)

# Show the figure
fig.show()

###Feature Importances Model

In [19]:
# Create feature importances
feature_importances = xgb_model.feature_importances_

# Sorting the feature importances
sorted_idx = np.argsort(feature_importances)[::-1]

# Creating a horizontal bar chart with Plotly
fig = go.Figure(go.Bar(
    x=feature_importances[sorted_idx],
    y=[X_train.columns[i] for i in sorted_idx],
    orientation='h',
    marker_color='rebeccapurple'
))

fig.update_layout(
    title='Feature Importances in XGBoost Model, Sorted by Importance',
    xaxis_title='Importance Score',
    yaxis=dict(autorange='reversed'),  # Invert y-axis to have the most important feature at the top
    width=1000,  # Width in pixels
    height=800   # Height in pixels
)

fig.show()

##Result

In [20]:
# Recoding the 'Predicted_Class' and 'Actual_Class'
X_test['Predicted_Class'] = X_test['Predicted_Conversion'].replace({0: 'Not Converted', 1: 'Converted'})
X_test['Actual_Class'] = X_test['Actual_Conversion'].replace({0: 'Not Converted', 1: 'Converted'})

# Resetting the index of X_test
X_test_reset = X_test.reset_index()

# Joining df1 with X_test_reset
result_df = df.join(X_test_reset, rsuffix='_drop', how='inner')

# Dropping columns from X_test that have the same names (these columns have '_drop' suffix)
result_df = result_df[[col for col in result_df.columns if not col.endswith('_drop')]]

# Dropping the 'Converted' column
result_df.drop('Converted', axis=1, inplace=True)

In [21]:
# Show DataFrame Leads Scoring result
result_df

,Prospect ID,Lead Number,Lead Origin,Lead Source,Do Not Email,Do Not Call,TotalVisits,Total Time Spent on Website,Page Views Per Visit,Last Activity,Country,Specialization,How did you hear about X Education,What is your current occupation,What matters most to you in choosing a course,Search,Magazine,Newspaper Article,X Education Forums,Newspaper,Digital Advertisement,Through Recommendations,Receive More Updates About Our Courses,Tags,Lead Quality,Update me on Supply Chain Content,Get updates on DM Content,Lead Profile,City,Asymmetrique Activity Index,Asymmetrique Profile Index,Asymmetrique Activity Score,Asymmetrique Profile Score,I agree to pay the amount through cheque,A free copy of Mastering The Interview,Last Notable Activity,Predicted_Probability,Probability_Category,Predicted_Conversion,Actual_Conversion,Predicted_Class,Actual_Class
0,7927b2df-8bba-4d29-b9a2-b6e0beafe620,660737,API,Olark Chat,No,No,0.0,0,0.00,Page Visited on Website,Missing,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Interested in other courses,Low in Relevance,No,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Modified,0.98,High Probability,1,1,Converted,Converted
1,2a272436-5132-4136-86fa-dcc88c88f482,660728,API,Organic Search,No,No,5.0,674,2.50,Email Opened,India,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Ringing,Missing,No,No,Select,Select,02.Medium,02.Medium,15.0,15.0,No,No,Email Opened,0.01,Undefined,0,0,Not Converted,Not Converted
2,8cc8c611-a219-4f35-ad23-fdfd2656bd8a,660727,Landing Page Submission,Direct Traffic,No,No,2.0,1532,2.00,Email Opened,India,Business Administration,Select,Student,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,Might be,No,No,Potential Lead,Mumbai,02.Medium,01.High,14.0,20.0,No,Yes,Email Opened,0.00,Undefined,0,0,Not Converted,Not Converted
3,0cc2df48-7cf4-4e39-9de9-19797f9b38cc,660719,Landing Page Submission,Direct Traffic,No,No,1.0,305,1.00,Unreachable,India,Media and Advertising,Word Of Mouth,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Ringing,Not Sure,No,No,Select,Mumbai,02.Medium,01.High,13.0,17.0,No,No,Modified,0.04,Undefined,0,0,Not Converted,Not Converted
4,3256f628-e534-4826-9d63-4a8b88782852,660681,Landing Page Submission,Google,No,No,2.0,1428,1.00,Converted to Lead,India,Select,Other,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,Might be,No,No,Select,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Modified,0.00,Undefined,0,0,Not Converted,Not Converted
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1843,e3d7f9af-1593-41e8-9682-c6ddfb9ee1b7,641612,Landing Page Submission,Direct Traffic,No,No,2.0,203,2.00,Email Opened,India,"Banking, Investment And Insurance",Missing,Missing,Missing,No,No,No,No,No,No,No,No,Missing,Missing,No,No,Missing,Mumbai,02.Medium,01.High,14.0,18.0,No,Yes,Email Opened,0.99,High Probability,1,1,Converted,Converted
1844,b3c94e33-69da-4163-a97e-a4825e233f95,641603,Landing Page Submission,Google,Yes,No,2.0,102,2.00,Email Bounced,India,Human Resource Management,Word Of Mouth,Missing,Missing,No,No,No,No,No,No,No,No,Missing,Missing,No,No,Missing,Mumbai,03.Low,01.High,12.0,18.0,No,No,Email Bounced,0.99,High Probability,1,1,Converted,Converted
1845,ff31e840-8fc7-45d0-a399-59accdb1f7dc,641602,API,Google,No,No,3.0,481,3.00,Email Opened,India,Human Resource Management,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Will revert after reading the email,High in Relevance,No,No,Potential Lead,Mumbai,02.Medium,01.High,15.0,18.0,No,No,Email Opened,0.00,Undefined,0,0,Not Converted,Not Converted
1846,1451a32b-16de-482e-825e-8cef8c017064,641585,API,Google,No,No,9.0,195,2.25,Unreachable,India,Select,Select,Unemployed,Better Career Prospects,No,No,No,No,No,No,No,No,Busy,Missing,No,No,Select,Select,02.Medium,02.Medium,